### Казюлина Марина
## Лабораторная работа №1
#### Версия 2.0

### Задание: 
Организовать хранение и поиск метаинформации о документах: 
- автор, 
- дата создания документа, 
- кто создал, 
- история правок (дата, кто правил),
- утверждения (дата, резолюция, чья резолюция). 

Хранение организовать в формате json.


### Мои уточнения
Кажется, задание не предполагает организацию ЗАПИСИ информации о метаданных документов, поэтому я сама сделала корпус метаданных некоторых документов, храню его в формате json и организую поиск по нему. Добавить документ и информацию о нем нельзя, это закрытый клуб придуманных мною документов.

Под документами здесь понимаются деловые бумаги, подтверждающие какой-нибудь факт или право на что-нибудь. 
Соответственно, резолюция - это надпись на документе, сделанная должностным лицом и содержащая принятое решение. 

Поиск всех метаданных организуется при помощи сообщения автора и даты его создания - это то, каким образом документы дифференцируются. У них всех, конечно, есть их собственный айди, но вспомнить имя и дату представляется более простой задачей, чем пятизначный код.

### Обновление 
#### Теперь есть возможность хранить и находить разные версии одного документа! 

#### Но как? 
Каждая версия любого документа имеет свой уникальный идентификатор, состоящий из пяти цифр. Метаданные каждой версии документа теперь включают в себя следующую информацию:
- от какой даты идет данная версия
- ее статус (актуален/неактуален)
- индекс оригинальной версии
- индекс последней версии 

Кроме этого, информация об индексах промежуточных версий хранятся в блоке правок! 

#### Хорошо, а искать-то как? 
Ну, во-первых, при вводе ФИО автора и даты создания документа находятся абсолютно все версии документа. 
Во-вторых, теперь также возможен поиск документа по индексу! 



Импортируем джейсон, достаем файл с метаданными, делаем storing_tree.
Storing_tree - это переменная, в которой лежит ВСЯ информация о документах, поиск внутри sorting_tree осуществляется только по ИНДЕКСУ. 

In [2]:
import json

In [3]:
with open('metadata2.json', 'r') as f:
    storing_tree = json.load(f)

In [4]:
for i in storing_tree.values():
    print(i, '\n')

{'id': '25994', 'author': 'Клышинский Эдуард Станиславович', 'issue date': '02.04.2004', 'status': {'version from': '23.09.2020', 'status': 'актуален', 'last index': '25994', 'original index': '25999'}, 'resolutions': [{'author': 'Клышинский Эдуард Станиславович', 'date': '03.04.2004', 'executive': 'Зотовой Л. А.', 'contents': 'Подготовить точку доступа мобильной связи для безболезненной трансляции в Zoom до 25.09.2020 9:29 '}], 'history': [{'author': 'Казюлина Марина Сергеевна', 'date of change': '23.09.2020', 'index': '25994'}]} 

{'id': '25999', 'author': 'Клышинский Эдуард Станиславович', 'issue date': '02.04.2004', 'status': {'version from': '02.09.2004', 'status': 'неактуален', 'last index': '25994', 'original index': '25999'}, 'resolutions': [{'author': 'Клышинский Эдуард Станиславович', 'date': '03.04.2004', 'executive': 'Зотовой Л. А.', 'contents': 'Подготовить точку доступа мобильной связи для безболезненной трансляции в Zoom до 25.09.2020 9:29 '}], 'history': [{'author': 'Ка

Делаем search_tree - переменную, в которой упорядоченно лежат ТОЛЬКО индексы, имена авторов и даты создания. Там нет остальных метаданных, и нужна эта переменная только для того, чтобы можно было найти индексы подходящих документов (или их версий) по имени автора и дате создания. Остальную информацию будем все равно брать из storing_tree

In [5]:
def formSearchTree(storing_tree):
    '''преобразуем корпус придуманных мною документов в структуру, по которой удобно производить поиск по а) имени б) дате'''
    search_tree = {}
    for doc in storing_tree.values():
        if doc['author'] not in search_tree:
            search_tree[doc['author']] = {doc['issue date'] : [doc['id']]}
        else:
            ''' если документ с таким автором уже был, то проверяем, 
            совпадают ли даты у этого документа и одного из тех, что у нас уже есть '''
            found = False
            for i in search_tree[doc['author']]:
                if doc['issue date'] == i:
                    ''' если даты совпали, до добавляем индекс документа в список индексов документов с таким же автором и датой'''
                    search_tree[doc['author']][i].append(doc['id'])
                    found = True
            if not found:
                search_tree[doc['author']][doc['issue date']] = [doc['id']]
                
    return search_tree
                                 

In [13]:
search_tree = formSearchTree(storing_tree)
print(search_tree)

{'Клышинский Эдуард Станиславович': {'02.04.2004': ['25994', '25999']}, 'Сериков Олег Алексеевич': {'03.05.2013': ['25995']}, 'Казюлина Марина Сергеевна': {'23.09.2020': ['75995', '95895', '95995'], '26.09.2020': ['20955']}, 'Путин Владимир Владимирович': {'26.09.2020': ['25955']}}


Создаем функции, которые будут 
а) общаться с пользователем
б) производить поиск
в) выводить информацию

Searching определяет индекс(ы) искомого документа по имени автора и дате создания при помощи search_tree и сообщает их printInfo. А printInfo уже в свою очередь обращается с полученными индексами к storing_tree и последовательно достает оттуда информацию о метаданных документов, после чего выводит их в консоль. 
Кроме этого, searching теперь может отличать индекс документа от имени автора, что позволяет сократить время поиска документа в случае, если пользователь владеет нужной информацией сам. 

In [14]:
def searching(search_tree):
    response = input('Если Вам известен индекс искомого документа, введите его, если нет - введите имя автора искомого документа: ')
    if response.isdigit():
        if response in storing_tree:
            printInfo([response])
        else:
            print('Документа с таким индексом не существует.')
    else:
        name = response
        if name in search_tree: 
            date = input('Введите дату создания искомого документа: ')
            if date in search_tree[name]:
                print(f'найден(о) {len(search_tree[name][date])} документ(ов)')
                if len(search_tree[name][date]):
                    printInfo(search_tree[name][date])
            else:
                print('Документов с такими параметрами не найдено.')
        else:
            print('Документов с такими параметрами не найдено. ')

        
def printInfo(lst):    
    for i in lst:
        line = f'''\nАвтор документа: {storing_tree[i]['author']}
Дата составления документа: {storing_tree[i]['issue date']}

    Версия документа от {storing_tree[i]['status']['version from']}
    Статус: {storing_tree[i]['status']['status']}
    Индекс оригинальной версии документа: {storing_tree[i]['status']['original index']}
    Индекс последней версии документа: {storing_tree[i]['status']['last index']}
        
    Резолюции: '''
        for res in storing_tree[i]['resolutions']:
            addition = f'''
        кому: {res['executive']}
        содержание:
        "{res['contents']}
        {res['author']}
        {res['date']}"
            '''
            line += addition
            
        if len(storing_tree[i]['history']):
            line += '\nПравки вносились следующими лицами в указанные даты:'

            for res in storing_tree[i]['history']:
                addition = f'''\n{res['author']} {res['date of change']}, индекс версии: {res['index']}'''
                line += addition
        else:
            addition = '\nПравок внесено не было.'
            line += addition
        
        print(line)
        

Тестируем:

In [8]:
searching(search_tree)

Если Вам известен индекс искомого документа, введите его, если нет - введите имя автора искомого документа: ддд
Введите имя автора искомого документа: ддд
Документов с такими параметрами не найдено. 


In [15]:
searching(search_tree)

Если Вам известен индекс искомого документа, введите его, если нет - введите имя автора искомого документа: Казюлина Марина Сергеевна
Введите дату создания искомого документа: 23.09.2020
найден(о) 3 документ(ов)

Автор документа: Казюлина Марина Сергеевна
Дата составления документа: 23.09.2020

    Версия документа от 25.09.2020
    Статус: актуален
    Индекс оригинальной версии документа: 95995
    Индекс последней версии документа: 75995
        
    Резолюции: 
        кому: Серикову О. А.
        содержание:
        "Уточнить ТЗ первой лабораторной работы до конца времени, отведенного на ее выполнение. 
        Казюлина Марина Сергеевна
        23.09.2020"
            
        кому: Путину В. В.
        содержание:
        "Починить дорогу у станции метро Севастопольская до 24.09.2020 
        Казюлина Марина Сергеевна
        23.09.2020"
            
Правки вносились следующими лицами в указанные даты:
Сериков Олег Алексеевич 24.09.2020, индекс версии: 95895
Казюлина Марина Серге

In [17]:
searching(search_tree)

Если Вам известен индекс искомого документа, введите его, если нет - введите имя автора искомого документа: Клышинский Эдуард Станиславович
Введите дату создания искомого документа: 02.04.2004
найден(о) 2 документ(ов)

Автор документа: Клышинский Эдуард Станиславович
Дата составления документа: 02.04.2004

    Версия документа от 23.09.2020
    Статус: актуален
    Индекс оригинальной версии документа: 25999
    Индекс последней версии документа: 25994
        
    Резолюции: 
        кому: Зотовой Л. А.
        содержание:
        "Подготовить точку доступа мобильной связи для безболезненной трансляции в Zoom до 25.09.2020 9:29 
        Клышинский Эдуард Станиславович
        03.04.2004"
            
Правки вносились следующими лицами в указанные даты:
Казюлина Марина Сергеевна 23.09.2020, индекс версии: 25994

Автор документа: Клышинский Эдуард Станиславович
Дата составления документа: 02.04.2004

    Версия документа от 02.09.2004
    Статус: неактуален
    Индекс оригинальной верси

In [20]:
searching(search_tree)

Если Вам известен индекс искомого документа, введите его, если нет - введите имя автора искомого документа: Путин Владимир Владимирович
Введите дату создания искомого документа: 26.09.2020
найден(о) 1 документ(ов)

Автор документа: Путин Владимир Владимирович
Дата составления документа: 26.09.2020

    Версия документа от 26.09.2020
    Статус: актуален
    Индекс оригинальной версии документа: 25955
    Индекс последней версии документа: 25955
        
    Резолюции: 
        кому: Медведеву М. А.
        содержание:
        "Купить хлеба по пути домой до 18:00 23.09.2020. 
        Путин Владимир Владимирович
        23.09.2020"
            
Правок внесено не было.


In [21]:
searching(search_tree)

Если Вам известен индекс искомого документа, введите его, если нет - введите имя автора искомого документа: 25995

Автор документа: Сериков Олег Алексеевич
Дата составления документа: 03.05.2013

    Версия документа от 03.05.2013
    Статус: актуален
    Индекс оригинальной версии документа: 25995
    Индекс последней версии документа: 25995
        
    Резолюции: 
        кому: Казюлиной М. С.
        содержание:
        "В случае отсутствия ответа на вопрос об уточнении ТЗ к 22:00 23.09.2020 пнуть пожалусто еще раз
        Сериков Олег Алексеевич
        23.09.2020"
            
Правок внесено не было.
